In [26]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [0]:
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

In [28]:
cd /content/drive/My Drive/Colab Notebooks/matrix/matrix_2/matryca_transformacja_1/data

/content/drive/My Drive/Colab Notebooks/matrix/matrix_2/matryca_transformacja_1/data


In [29]:
df = pd.read_hdf('car.h5')
df.shape

(106494, 155)

In [0]:
SUFFIX_CAT = '__cat'


for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = df[feat].factorize()[0]

In [0]:
def run_model(model, feats):
  x = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, x, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))
df['param_moc'] = df['param_moc'].map(lambda x:  1 if str(x) == 'None' else int(x.split(' ')[0]))
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x:  1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ','')))

In [33]:
feats = ['param_napęd__cat', 'param_rok-produkcji','param_stan__cat' ,'param_skrzynia-biegów__cat','param_faktura-vat__cat',
'param_moc', 'param_marka-pojazdu__cat',
'feature_kamera-cofania__cat',
'param_typ__cat',
'seller_name__cat',
'param_pojemność-skokowa',
'feature_wspomaganie-kierownicy__cat',
'param_model-pojazdu__cat',
'param_wersja__cat',
'param_kod-silnika__cat', 
'feature_system-start-stop__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_regulowane-zawieszenie__cat',]
len(feats)

20

In [34]:
xgb_params = {
    'max_depth' :5, 
    'n_estimators': 50, 
    'learning_rate' : 0.1,
    'seed': 0
}
run_model(xgb.XGBRegressor(**xgb_params), feats)

[15:52:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:52:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:52:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9564.58895203383, 73.0539676664732)

In [37]:
def obj_func(params):
  print("Training with params: ")
  print(params)

  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)

  return {'loss': np.abs(mean_mae), 'status' : STATUS_OK}


  #space 
xgb_reg_params = {
      'learning_rate': hp.choice('learning_rate', np.arange(0.05, 0.31, 0.05)),
      'max_depth': hp.choice('max_depth', np.arange(5,16,1, dtype=int)),
      'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
      'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.5),
      'objective': 'reg:squarederror',
      'n_estimators': 100,
      'seed': 0,
}

best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)
best

Training with params: 
{'colsample_bytree': 1.0, 'learning_rate': 0.15000000000000002, 'max_depth': 12, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8}
Training with params: 
{'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 15, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params: 
{'colsample_bytree': 0.5, 'learning_rate': 0.1, 'max_depth': 15, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.6000000000000001}
Training with params: 
{'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 14, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params: 
{'colsample_bytree': 1.0, 'learning_rate': 0.25, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params: 
{'colsample_bytree': 1.0, 'learning_rate': 0.15000000000000002, 'max_depth': 11, 'n_esti

{'colsample_bytree': 1.0, 'learning_rate': 2, 'max_depth': 7, 'subsample': 0.9}